In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

def import_sys():
    import sys
    sys.path.append('..')
import_sys()

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)  # pylint: disable=invalid-name

In [2]:
import errudite
print(errudite)

<module 'errudite' from '/Users/gyuhoshim/errudite/errudite/__init__.py'>


In [3]:
import pandas as pd
from tqdm import tqdm

from overrides import overrides

from errudite.io import DatasetReader
from errudite.utils import normalize_file_path, accuracy_score
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from errudite.targets.label import Label, PredefinedLabel

In [4]:
#!pip install transformers==4.3.0

In [5]:
#!pip install sentencepiece

In [6]:
#!pip install accelerate

In [7]:
#!pip install  xlrd==1.1.0

In [8]:
#!pip install --upgrade ipywidgets

In [9]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from typing import List, Dict
import sys
sys.path.append('..')
from errudite.predictors.predictor import Predictor
from errudite.targets.label import Label, PredefinedLabel

class Predictor_FLAN_T5(Predictor):
    def __init__(self, name: str, 
    model_path: str=None,
    model_online_path: str=None,
    description: str='',
    model_type: str=None) -> None:
        model = None
        Predictor.__init__(self, name, description, model, ['accuracy', 'accuracy_PE_Acceptable', 'accuracy_PE_Unacceptable', 'accuracy_PE_Insufficient', 'accuracy_PE_NotFound'
                                                            ,'accuracy_KE_Acceptable', 'accuracy_KE_Unacceptable', 'accuracy_KE_Insufficient', 'accuracy_KE_NotFound',
                                                           'accuracy_LCE_Acceptable', 'accuracy_LCE_Unacceptable', 'accuracy_LCE_Insufficient', 'accuracy_LCE_NotFound'])
        
    def predict_essay(self, essay : str) -> Dict[str,str]:
        # Load data into pandas DataFrame
        data_df = pd.read_excel("StudentEssays.xlsx")

        # Initialize T5 tokenizer and model
        tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
        model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

        # Move the model to the CUDA device if available
        if torch.cuda.is_available():
            model.to("cuda")

        # Define a list of concepts to predict
        concepts_to_predict = ["potential energy", "kinetic energy", "Law of Conservation of Energy"]

        # Define possible outcome labels
        outcome_labels = ["Acceptable", "Unacceptable", "Insufficient", "Not Found"]

        # Create a list to store predictions as dictionaries
        predictions_list = []

        text = essay  # Assuming the text content is in column 'Essay'

        # Initialize predictions dictionary for this row
        predictions = {}

        # Iterate through each concept to predict
        for concept in concepts_to_predict:
            # Define a template for classification
            template = f"According to the following essay, is the student's definition of {concept} Acceptable, Unacceptable, Insufficient, or Not Found? Only use one of these labels for outputs\n{text}"
            # Prepare the input by replacing placeholders
            formatted_input = template
            # Tokenize and classify the text
            input_ids = tokenizer(formatted_input, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model.generate(input_ids, max_length=128)
            decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Remove special tokens

            # Store the prediction in the dictionary
            predictions[concept] = next((label for label in outcome_labels if label.lower() in decoded_output.lower()), "Unknown")

            if predictions[concept] == "Unknown":
              print(len(decoded_output))
              with open('output.txt', 'w') as f:
                f.write(decoded_output)

        return predictions

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_transformers.modeling_bert:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_transformers.modeling_xlnet:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>


## 1. DatasetReader

In [10]:
@DatasetReader.register("STE")
class STEReader(DatasetReader):
    def __init__(self, cache_folder_path: str=None) -> None:
        super().__init__(cache_folder_path)
        # overwrite the primary evaluation method and metric name
        Label.set_task_evaluator(accuracy_score, 'accuracy')
        
    @overrides
    def _read(self, file_path: str, lazy: bool, sample_size: int):
        """
        Returns a list containing all the instances in the specified dataset.
 
        Parameters
        ----------
        file_path : str
            The path of the input data file.
        lazy : bool, optional
            If ``lazy==True``, only run the tokenization, does not compute the linguistic
            features like POS, NER. By default False
        sample_size : int, optional
            If sample size is set, only load this many of instances, by default None
        
        Returns
        -------
        List[Instance]
            The instance list.
        """
        instances = []
        essays = []
        logger.info("Reading instances from lines in file at: %s", file_path)
        df = pd.read_excel(normalize_file_path(file_path), sep='\t')
        for idx, row in tqdm(df.iterrows()):
            if lazy:
                essays.append(row['Essay'])
            else:
                instance = self._text_to_instance(f'q:{idx}', row)
                if instance is not None:
                    instances.append(instance)
                if sample_size and idx > sample_size:
                    break
        if lazy:
            return { "Essays": essays }
        else:
            return instances
    
    @overrides
    def _text_to_instance(self, id: str, row) -> Instance:
        # The function that transfers raw text to instance.
        essay = Target(qid=row['Essay_ID'], text=row['Essay'], vid=0, metas={'type': 'essays'})        # label
        groundtruth_PE = PredefinedLabel(
            model='groundtruth', 
            qid=row['Essay_ID'], 
            text=row['PE'], 
            vid=0, 
        )
        groundtruth_KE = PredefinedLabel(
            model='groundtruth', 
            qid=row['Essay_ID'], 
            text=row['KE'], 
            vid=0, 
        )
        groundtruth_LCE = PredefinedLabel(
            model='groundtruth', 
            qid=row['Essay_ID'], 
            text=row['LCE'], 
            vid=0, 
        )
        return self.create_instance(row['Essay_ID'], 
            essay=essay, 
            groundtruth_PE=groundtruth_PE,
            groundtruth_KE=groundtruth_KE,
            groundtruth_LCE=groundtruth_LCE)

In [11]:
from errudite.io import DatasetReader

cache_folder_path = "./ste_caches"
reader = DatasetReader.by_name("STE")(cache_folder_path=cache_folder_path)

INFO:errudite.utils.file_utils:Errudite cache folder selected: ./ste_caches


In [12]:
# read the raw data!
instances = reader.read(
    # The path of the input data file. We are using the first 100 rows from the SNLI dev set.
    file_path='StudentEssays.xlsx', 
    # If sample size is set, only load this many of instances, by default None.
    sample_size=76)

INFO:errudite.io.dataset_reader:Reading instances from lines in file at: StudentEssays.xlsx
INFO:__main__:Reading instances from lines in file at: StudentEssays.xlsx
76it [00:02, 30.09it/s]


In [13]:
#instances

## 2. Predictor

In [14]:
from typing import List, Dict
import sys
sys.path.append('..')
from errudite.predictors.predictor import Predictor
from errudite.targets.label import Label, PredefinedLabel

In [15]:
@Predictor.register("STE_FLAN_T5")
class PredictorSTE(Predictor_FLAN_T5):
    def __init__(self, name: str, 
        model_path: str=None,
        model_online_path: str=None,
        description: str='') -> None:
        Predictor_FLAN_T5.__init__(self, name, model_path, model_online_path, description)
        import sys
        sys.path.append('..')
        from errudite.utils.evaluator import accuracy_score
        # Second, from the metrics above, pick one that's primary, and it will be used 
        # to compute `is_incorrect()` in any label target object: primary metric < 1.
        Label.set_task_evaluator(
            # the evaluation function that accepts pred and groundtruths, 
            # and return a dict of metrics: { metric_name: metric_score }. 
            # This is saved as Label.task_evaluation_func.
            task_evaluation_func=accuracy_score, 
            # The primary task metric name, ideally a key of task_evaluation_func ‘s return.
            task_primary_metric='accuracy')

    # the raw prediction function, returning the output of the model in a json format.
    def predict(self, essay: str) -> Dict[str, str]:
        predicted = self.predict_essay(essay)
        return predicted

    @classmethod
    # the class method that takes `Target` inputs, and output a `Label` object.
    def model_predict(cls, 
        predictor: Predictor, 
        essay: Target, 
        groundtruth_PE: Label, groundtruth_KE: Label, groundtruth_LCE: Label) -> 'Label':
        answer = None
        if not predictor:
            return answer
        predicted = predictor.predict(essay.get_text())
        if not predicted:
            return None
        answer_PE = PredefinedLabel(
            model=predictor.name, 
            qid=essay.qid,
            text=predicted['potential energy'], 
            vid=max([essay.vid]))
        answer_KE = PredefinedLabel(
            model=predictor.name, 
            qid=essay.qid,
            text=predicted['kinetic energy'], 
            vid=max([essay.vid]))
        answer_LCE = PredefinedLabel(
            model=predictor.name, 
            qid=essay.qid,
            text=predicted['Law of Conservation of Energy'], 
            vid=max([essay.vid]))
        answer_PE.compute_perform(groundtruths=groundtruth_PE)
        answer_KE.compute_perform(groundtruths=groundtruth_KE)
        answer_LCE.compute_perform(groundtruths=groundtruth_LCE)
        return answer_PE, answer_KE, answer_LCE



In [16]:
from errudite.predictors import Predictor
model_path = "FLAN_T5_Essay"
predictor = Predictor.by_name("STE_FLAN_T5")(
    name ='FLAN_T5', 
    description='Prediction created by FLAN_T5')

In [ ]:
from tqdm import tqdm 
logger.info("Running predictions....") 
for instance in tqdm(instances):
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = instance.essay,
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # set the prediction
    instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)

INFO:__main__:Running predictions....
 82%|████████▏ | 62/76 [18:07<09:06, 39.01s/it]

In [ ]:
print(instances[2].get_entry('essay'), "\n")
instances[2].show_instance()

In [ ]:
predictor.evaluate_performance(instances)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
# ---------
# Build the instance store hash
from errudite.targets.instance import Instance
instance_hash, instance_hash_rewritten, qid_hash = Instance.build_instance_hashes(instances)

In [ ]:
print(instances[2].is_incorrect(model = 'FLAN_T5'), "\n")

In [ ]:
from errudite.builts import Group
from errudite.builts import Attribute
group_length_10 = Group.create(
    # The name of the attribute
    name="length",
    # the description of the attribute
    description="length greater than 10",
    # All the previously created attributes and groups 
    # can be used and queried, as long as we serve the 
    # stored attributes and groups as part of the inputs.
    cmd="length(essay) > 10",
    attr_hash=Attribute.store_hash(),
    group_hash=Group.store_hash()
)

In [ ]:
group_length_10.visualize_models(models=["FLAN_T5"])

In [ ]:
# some general stats on groups
Group.eval_stats(
    filtered_instances=group_length_10.get_instances(),
    # this will automatically call the default model we got
    model=None
)

In [ ]:
from errudite.builts import Group
from errudite.builts import Attribute
group_length_200 = Group.create(
    # The name of the attribute
    name="length",
    # the description of the attribute
    description="length greater than 200",
    # All the previously created attributes and groups 
    # can be used and queried, as long as we serve the 
    # stored attributes and groups as part of the inputs.
    cmd="length(essay) > 200",
    attr_hash=Attribute.store_hash(),
    group_hash=Group.store_hash()
)

In [ ]:
group_length_200.visualize_models(models=["FLAN_T5"])

In [ ]:
# some general stats on groups
Group.eval_stats(
    filtered_instances=group_length_200.get_instances(),
    # this will automatically call the default model we got
    model=None
)

REWRITES!!

In [ ]:
from errudite.rewrites import Rewrite
rewrite_rest_motion = Rewrite.create_with_cmd(
    from_cmd='rest', 
    to_cmd= 'motion', 
    target_cmd= 'essay'
)

In [ ]:
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from tqdm import tqdm

rewrite_rest_var=[]

for key in tqdm(group_length_10.get_instances()):
#for key in tqdm(instances):
    """
    This function returns a named tuple, with rid and the 
    rewritten text of the instance intended target. 
    If the instance cannot be rewritten, return None.
    """
    instance = Instance.get(key)
    rewritten_output = rewrite_rest_motion.rewrite_one_instance(instance)
    if not rewritten_output:
        rewrite_rest_var.append(instance)
        continue
    entries = {}
    # compute the vid automatically
    vid = len(Instance.qid_hash[instance.qid])
    # init the instance 
    rewritten_instance = Instance(
        qid=instance.qid, vid=vid, rid=rewrite_rest_motion.rid)
    for e in instance.entries:
        if e in rewrite_rest_motion.target_cmd:
            # automatically create a new Target for the 
            # rewritten part.
            entry = Target(
                qid=instance.qid, 
                text=rewritten_output.text, 
                vid=vid, 
                metas=instance.get_entry(e).metas)
        else:
            # Otherwise, use the original target
            entry = instance.get_entry(e)
        # save all the entries
        rewritten_instance.set_entries(**{e: entry})
    # run the prediction
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = rewritten_instance.get_entry('essay'),
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # save the prediction
    rewritten_instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)
    rewrite_rest_var.append(rewritten_instance)
    # set the rewritten key into the rewrite
    rewrite_rest_motion.add_instance(rewritten_instance.key())
    # save into the Instance hashes
    #print(rewritten_instance)
    Instance.save(rewritten_instance)

In [ ]:
predictor.evaluate_performance(instances)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
predictor.evaluate_performance(rewrite_rest_var)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
from errudite.rewrites import Rewrite
rewrite_ke = Rewrite.create_with_cmd(
    from_cmd='kinetic energy', 
    to_cmd= '#', 
    target_cmd= 'essay'
)

In [ ]:
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from tqdm import tqdm

rewrite_ke_var=[]

for key in tqdm(group_length_10.get_instances()):
#for key in tqdm(instances):
    """
    This function returns a named tuple, with rid and the 
    rewritten text of the instance intended target. 
    If the instance cannot be rewritten, return None.
    """
    instance = Instance.get(key)
    rewritten_output = rewrite_ke.rewrite_one_instance(instance)
    if not rewritten_output:
        rewrite_ke_var.append(instance)
        continue
    entries = {}
    # compute the vid automatically
    vid = len(Instance.qid_hash[instance.qid])
    # init the instance 
    rewritten_instance = Instance(
        qid=instance.qid, vid=vid, rid=rewrite_ke.rid)
    for e in instance.entries:
        if e in rewrite_ke.target_cmd:
            # automatically create a new Target for the 
            # rewritten part.
            entry = Target(
                qid=instance.qid, 
                text=rewritten_output.text, 
                vid=vid, 
                metas=instance.get_entry(e).metas)
        else:
            # Otherwise, use the original target
            entry = instance.get_entry(e)
        # save all the entries
        rewritten_instance.set_entries(**{e: entry})
    # run the prediction
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = rewritten_instance.get_entry('essay'),
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # save the prediction
    rewritten_instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)
    rewrite_ke_var.append(rewritten_instance)
    # set the rewritten key into the rewrite
    rewrite_ke.add_instance(rewritten_instance.key())
    # save into the Instance hashes
    #print(rewritten_instance)
    Instance.save(rewritten_instance)

In [ ]:
predictor.evaluate_performance(instances)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
predictor.evaluate_performance(rewrite_ke_var)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
#rewrite_rest_motion.visualize_models(models=["FLAN_T5"])

In [ ]:
from errudite.rewrites import Rewrite
rewrite_top_bottom = Rewrite.create_with_cmd(
    from_cmd='top', 
    to_cmd= 'bottom', 
    target_cmd= 'essay'
)

In [ ]:
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from tqdm import tqdm

rewrite_tb_var=[]

for key in tqdm(group_length_10.get_instances()):
    """
    This function returns a named tuple, with rid and the 
    rewritten text of the instance intended target. 
    If the instance cannot be rewritten, return None.
    """
    instance = Instance.get(key)
    rewritten_output = rewrite_top_bottom.rewrite_one_instance(instance)
    if not rewritten_output:
        rewrite_tb_var.append(instance)
        continue
    entries = {}
    # compute the vid automatically
    vid = len(Instance.qid_hash[instance.qid])
    # init the instance 
    rewritten_instance = Instance(
        qid=instance.qid, vid=vid, rid=rewrite_top_bottom.rid)
    for e in instance.entries:
        if e in rewrite_top_bottom.target_cmd:
            # automatically create a new Target for the 
            # rewritten part.
            entry = Target(
                qid=instance.qid, 
                text=rewritten_output.text, 
                vid=vid, 
                metas=instance.get_entry(e).metas)
        else:
            # Otherwise, use the original target
            entry = instance.get_entry(e)
        # save all the entries
        rewritten_instance.set_entries(**{e: entry})
    # run the prediction
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = rewritten_instance.get_entry('essay'),
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # save the prediction
    rewritten_instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)
    rewrite_tb_var.append(rewritten_instance)
    # set the rewritten key into the rewrite
    rewrite_top_bottom.add_instance(rewritten_instance.key())
    
    # save into the Instance hashes
    Instance.save(rewritten_instance)

In [ ]:
predictor.evaluate_performance(instances)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
predictor.evaluate_performance(rewrite_tb_var)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
from errudite.rewrites import Rewrite
rewrite_car_stone = Rewrite.create_with_cmd(
    from_cmd='car', 
    to_cmd= 'stone', 
    target_cmd= 'essay'
)

In [ ]:
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from tqdm import tqdm

rewrite_car_var=[]

for key in tqdm(group_length_10.get_instances()):
    """
    This function returns a named tuple, with rid and the 
    rewritten text of the instance intended target. 
    If the instance cannot be rewritten, return None.
    """
    instance = Instance.get(key)
    rewritten_output = rewrite_car_stone.rewrite_one_instance(instance)
    if not rewritten_output:
        rewrite_car_var.append(instance)
        continue
    entries = {}
    # compute the vid automatically
    vid = len(Instance.qid_hash[instance.qid])
    # init the instance 
    rewritten_instance = Instance(
        qid=instance.qid, vid=vid, rid=rewrite_car_stone.rid)
    for e in instance.entries:
        if e in rewrite_car_stone.target_cmd:
            # automatically create a new Target for the 
            # rewritten part.
            entry = Target(
                qid=instance.qid, 
                text=rewritten_output.text, 
                vid=vid, 
                metas=instance.get_entry(e).metas)
        else:
            # Otherwise, use the original target
            entry = instance.get_entry(e)
        # save all the entries
        rewritten_instance.set_entries(**{e: entry})
    # run the prediction
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = rewritten_instance.get_entry('essay'),
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # save the prediction
    rewritten_instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)
    rewrite_car_var.append(rewritten_instance)
    # set the rewritten key into the rewrite
    rewrite_car_stone.add_instance(rewritten_instance.key())
    
    # save into the Instance hashes
    Instance.save(rewritten_instance)

In [ ]:
predictor.evaluate_performance(instances)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
predictor.evaluate_performance(rewrite_car_var)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
from errudite.rewrites import Rewrite
rewrite_not_blank = Rewrite.create_with_cmd(
    from_cmd='not', 
    to_cmd= ' ', 
    target_cmd= 'essay'
)

In [ ]:
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from tqdm import tqdm

rewrite_not_var=[]

for key in tqdm(group_length_10.get_instances()):
    """
    This function returns a named tuple, with rid and the 
    rewritten text of the instance intended target. 
    If the instance cannot be rewritten, return None.
    """
    instance = Instance.get(key)
    rewritten_output = rewrite_not_blank.rewrite_one_instance(instance)
    if not rewritten_output:
        rewrite_not_var.append(instance)
        continue
    entries = {}
    # compute the vid automatically
    vid = len(Instance.qid_hash[instance.qid])
    # init the instance 
    rewritten_instance = Instance(
        qid=instance.qid, vid=vid, rid=rewrite_not_blank.rid)
    for e in instance.entries:
        if e in rewrite_not_blank.target_cmd:
            # automatically create a new Target for the 
            # rewritten part.
            entry = Target(
                qid=instance.qid, 
                text=rewritten_output.text, 
                vid=vid, 
                metas=instance.get_entry(e).metas)
        else:
            # Otherwise, use the original target
            entry = instance.get_entry(e)
        # save all the entries
        rewritten_instance.set_entries(**{e: entry})
    # run the prediction
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = rewritten_instance.get_entry('essay'),
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # save the prediction
    rewritten_instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)
    rewrite_not_var.append(rewritten_instance)
    # set the rewritten key into the rewrite
    rewrite_not_blank.add_instance(rewritten_instance.key())
    
    # save into the Instance hashes
    Instance.save(rewritten_instance)

In [ ]:
predictor.evaluate_performance(instances)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
predictor.evaluate_performance(rewrite_not_var)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
from errudite.rewrites import Rewrite
rewrite_is = Rewrite.create_with_cmd(
    from_cmd='is', 
    to_cmd= 'is not', 
    target_cmd= 'essay'
)

In [ ]:
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from tqdm import tqdm

rewrite_is_var=[]

for key in tqdm(group_length_10.get_instances()):
    """
    This function returns a named tuple, with rid and the 
    rewritten text of the instance intended target. 
    If the instance cannot be rewritten, return None.
    """
    instance = Instance.get(key)
    rewritten_output = rewrite_is.rewrite_one_instance(instance)
    if not rewritten_output:
        rewrite_is_var.append(instance)
        continue
    entries = {}
    # compute the vid automatically
    vid = len(Instance.qid_hash[instance.qid])
    # init the instance 
    rewritten_instance = Instance(
        qid=instance.qid, vid=vid, rid=rewrite_is.rid)
    for e in instance.entries:
        if e in rewrite_is.target_cmd:
            # automatically create a new Target for the 
            # rewritten part.
            entry = Target(
                qid=instance.qid, 
                text=rewritten_output.text, 
                vid=vid, 
                metas=instance.get_entry(e).metas)
        else:
            # Otherwise, use the original target
            entry = instance.get_entry(e)
        # save all the entries
        rewritten_instance.set_entries(**{e: entry})
    # run the prediction
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = rewritten_instance.get_entry('essay'),
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # save the prediction
    rewritten_instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)
    rewrite_is_var.append(rewritten_instance)
    # set the rewritten key into the rewrite
    rewrite_is.add_instance(rewritten_instance.key())
    
    # save into the Instance hashes
    Instance.save(rewritten_instance)

In [ ]:
predictor.evaluate_performance(instances)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
predictor.evaluate_performance(rewrite_is_var)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
from errudite.rewrites import Rewrite
rewrite_created = Rewrite.create_with_cmd(
    from_cmd='created', 
    to_cmd= '#', 
    target_cmd= 'essay'
)

In [ ]:
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from tqdm import tqdm

rewrite_created_var=[]

for key in tqdm(group_length_10.get_instances()):
    """
    This function returns a named tuple, with rid and the 
    rewritten text of the instance intended target. 
    If the instance cannot be rewritten, return None.
    """
    instance = Instance.get(key)
    rewritten_output = rewrite_created.rewrite_one_instance(instance)
    if not rewritten_output:
        rewrite_created_var.append(instance)
        continue
    entries = {}
    # compute the vid automatically
    vid = len(Instance.qid_hash[instance.qid])
    # init the instance 
    rewritten_instance = Instance(
        qid=instance.qid, vid=vid, rid=rewrite_created.rid)
    for e in instance.entries:
        if e in rewrite_created.target_cmd:
            # automatically create a new Target for the 
            # rewritten part.
            entry = Target(
                qid=instance.qid, 
                text=rewritten_output.text, 
                vid=vid, 
                metas=instance.get_entry(e).metas)
        else:
            # Otherwise, use the original target
            entry = instance.get_entry(e)
        # save all the entries
        rewritten_instance.set_entries(**{e: entry})
    # run the prediction
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = rewritten_instance.get_entry('essay'),
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # save the prediction
    rewritten_instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)
    rewrite_created_var.append(rewritten_instance)
    # set the rewritten key into the rewrite
    rewrite_created.add_instance(rewritten_instance.key())
    
    # save into the Instance hashes
    Instance.save(rewritten_instance)

In [ ]:
predictor.evaluate_performance(instances)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
predictor.evaluate_performance(rewrite_created_var)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
from errudite.rewrites import Rewrite
rewrite_moving = Rewrite.create_with_cmd(
    from_cmd='moving', 
    to_cmd= ' ', 
    target_cmd= 'essay'
)

In [ ]:
from errudite.targets.instance import Instance
from errudite.targets.target import Target
from tqdm import tqdm

rewrite_moving_var=[]

for key in tqdm(group_length_10.get_instances()):
    """
    This function returns a named tuple, with rid and the 
    rewritten text of the instance intended target. 
    If the instance cannot be rewritten, return None.
    """
    instance = Instance.get(key)
    rewritten_output = rewrite_moving.rewrite_one_instance(instance)
    if not rewritten_output:
        rewrite_moving_var.append(instance)
        continue
    entries = {}
    # compute the vid automatically
    vid = len(Instance.qid_hash[instance.qid])
    # init the instance 
    rewritten_instance = Instance(
        qid=instance.qid, vid=vid, rid=rewrite_moving.rid)
    for e in instance.entries:
        if e in rewrite_moving.target_cmd:
            # automatically create a new Target for the 
            # rewritten part.
            entry = Target(
                qid=instance.qid, 
                text=rewritten_output.text, 
                vid=vid, 
                metas=instance.get_entry(e).metas)
        else:
            # Otherwise, use the original target
            entry = instance.get_entry(e)
        # save all the entries
        rewritten_instance.set_entries(**{e: entry})
    # run the prediction
    prediction_PE, prediction_KE, prediction_LCE = Predictor.by_name("STE_FLAN_T5").model_predict(
        predictor, 
        essay = rewritten_instance.get_entry('essay'),
        groundtruth_PE = instance.groundtruth_PE, groundtruth_KE = instance.groundtruth_KE, groundtruth_LCE = instance.groundtruth_LCE)
    # save the prediction
    rewritten_instance.set_entries(prediction_PE = prediction_PE, prediction_KE = prediction_KE, prediction_LCE = prediction_LCE)
    rewrite_moving_var.append(rewritten_instance)
    # set the rewritten key into the rewrite
    rewrite_moving.add_instance(rewritten_instance.key())
    
    # save into the Instance hashes
    Instance.save(rewritten_instance)

In [ ]:
predictor.evaluate_performance(rewrite_moving_var)
print({"predictor": predictor.name, "perform": predictor.perform })

In [ ]:
import traceback
from typing import Union, List
from spacy.tokens import Doc, Span, Token
def import_sys():
    import sys
    sys.path.append('..')
import_sys()
from errudite.utils.helpers import convert_doc
from errudite.utils.check import DSLValueError
import logging
logger = logging.getLogger(__name__)  # pylint: disable=invalid-name


In [ ]:
from errudite.build_blocks import PrimFunc
from errudite.build_blocks.prim_funcs.linguistic import linguistic

@PrimFunc.register()
def contains_quantity(docs: Union['Target', Span]) -> bool:
    """
    Detect the presence of quantity entities in the essay.
    """
    # Use the linguistic function to extract entity types
    entities = linguistic(docs, label='ent_type')
    #print(entities)
    contains='QUANTITY' in entities
    # Check if 'bottom' or 'top' is present in the extracted entity types
    #print(contains)
    return contains


In [ ]:
from errudite.builts import Attribute, Group

# Create an attribute based on the location function
attr = Attribute.create(
    name="quantity_entities",
    description="Presence of location entities in the essay",
    cmd="location(essay)"
)

# Create a group that checks for the presence of location entities
quantity_group = Group.create(
    name="quantity",
    description="quantity entity detected",
    cmd="attr:quantity_entities == TRUE",
    attr_hash=Attribute.store_hash(),
    group_hash=Group.store_hash()
)
quantity_group

In [ ]:
# some general stats on groups
Group.eval_stats(
    filtered_instances=quantity_group.get_instances(),
    # this will automatically call the default model we got
    model=None
)

In [ ]:
quantity_group.visualize_models(models=["FLAN_T5"])

In [ ]:
from errudite.build_blocks import PrimFunc
from errudite.build_blocks.prim_funcs.linguistic import STRING
import spacy
nlp = spacy.load("en_core_web_sm")


@PrimFunc.register()
def num_adjectives(target: 'Target') -> int:
    """
    Count the number of adjectives in a given target.
    """
    try:
        # Access the tokens associated with the Target
        #print(STRING(target))
        #tokens = STRING(target).tokens
        doc = nlp(STRING(target))
        adjectives = [token for token in doc if token.pos_ == "ADJ"]
        return len(adjectives)
    except Exception as e:
        ex = Exception(f"Unknown exception from [num_adjectives]: {e}")
        raise ex




In [ ]:
from errudite.builts import Attribute, Group

# Assuming you have already defined the PrimFunc num_adjectives

# Create an attribute that counts the adjectives among essay targets
attr = Attribute.create(
    name="num_adjectives_in_essay",
    description="Number of adjectives among essay targets",
    cmd="num_adjectives(essay)"
)


# Create a group to check if there are more than 5 adjectives in the essay
adj_count_group = Group.create(
    name="adjective_count_group",
    description="Group for counting adjectives in the essay",
    cmd="attr:num_adjectives_in_essay > 10",
    attr_hash=Attribute.store_hash(),
    group_hash=Group.store_hash()
)

# You can now use the 'group' to check if there are more than 5 adjectives in your essay targets.
adj_count_group

In [ ]:
# some general stats on groups
Group.eval_stats(
    filtered_instances=adj_count_group.get_instances(),
    # this will automatically call the default model we got
    model=None
)

In [ ]:
adj_count_group.visualize_models(models=["FLAN_T5"])

In [ ]:
from errudite.build_blocks import PrimFunc
from errudite.build_blocks.prim_funcs.linguistic import STRING
import spacy
nlp = spacy.load("en_core_web_sm")


@PrimFunc.register()
def contains_loc(target: 'Target') -> int:
    """
    Detect the presence of location entities ('bottom' or 'top') in the essay.
    """
    try:
        # Access the tokens associated with the Target
        target_str=STRING(target)
        #tokens = STRING(target).tokens
        if "bottom" in target_str or "top" in target_str:
            return True
    except Exception as e:
        ex = Exception(f"Unknown exception from [num_adjectives]: {e}")
        raise ex



In [ ]:
from errudite.builts import Attribute, Group

# Assuming you have already defined the PrimFunc num_adjectives

# Create an attribute that counts the adjectives among essay targets
attr = Attribute.create(
    name="contains_loc_in_essay",
    description="Number of adjectives among essay targets",
    cmd="contains_loc(essay)"
)


# Create a group to check if there are more than 5 adjectives in the essay
contains_loc_group = Group.create(
    name="adjective_count_group",
    description="Group for counting adjectives in the essay",
    cmd="attr:contains_loc_in_essay==TRUE ",
    attr_hash=Attribute.store_hash(),
    group_hash=Group.store_hash()
)

# You can now use the 'group' to check if there are more than 5 adjectives in your essay targets.
contains_loc_group

In [ ]:
# some general stats on groups
Group.eval_stats(
    filtered_instances=contains_loc_group.get_instances(),
    # this will automatically call the default model we got
    model=None
)

In [ ]:
contains_loc_group.visualize_models(models=["FLAN_T5"])